In [102]:
import pandas as pd
import seaborn as sns
import numpy as np

In [103]:
%cd '~/git/nogitfile'
!ls
ori_df = pd.read_csv('train.csv')

### 看有幾個column
print(f'-共有{ori_df.shape[0]}筆資料')
#print('-共有{0}筆資料'.format(df.shape[0]))
print('-分布於%d欄'%ori_df.shape[1])
import numpy as np
def timeSinCosTrans(df):

    Time = df.loc[:,'loctm'].values
    # // 是整除的意思
    s, m, h = [(Time//100**(i)%100) for i in range(3)]
    Time = h*3600+m*60+s

    dfTime = pd.DataFrame()
    dfTime.loc[:,'TIME_SIN'] = np.sin(2*np.pi*Time/(24*60*60))
    dfTime.loc[:,'TIME_COS'] = np.cos(2*np.pi*Time/(24*60*60))
    dfTime.loc[:,'time'] = Time

    #df.drop(columns = 'loctm', inplace =True)
    df = pd.concat([df, dfTime], axis = 1)

    return df

/home/jovyan/git/nogitfile
dataset_description.pdf  submit_11-07:15:54.csv  train.csv
submit_11-07:14:04.csv	 test.csv
-共有1521787筆資料
-分布於23欄


In [104]:
def submit():

    test_df = pd.read_csv('test.csv')
    #rint(len(test_df))
    ID = test_df['txkey']


   # to_drop = ['bacno','txkey','cano']
   # test_df = test_df.drop(columns=to_drop)

    #print(test_df.info())
    test_df = timeSinCosTrans(test_df)

   # print(len(test_df))
    test_df.fillna(0, inplace = True)

    test = test_df

    YN = ['ecfg','flbmk', 'flg_3dsmk', 'insfg', 'ovrlt']
    for col in YN:
        test.loc[test[col] =='N', col] = 0
        test.loc[test[col] =='Y', col] = 1


    

    label = model.predict(test)

    from datetime import datetime
    time_str = datetime.now().strftime('%m-%d:%H:%M')

    submit = {'txkey':ID, 'fraud_ind':label}
    pd.DataFrame(data= submit).to_csv('submit_'+time_str+'.csv',index = False)

### 基礎資料處理

In [105]:
df = ori_df.copy()

# 把Y/N 資料轉成 0/1
YN = ['ecfg','flbmk', 'flg_3dsmk', 'insfg', 'ovrlt']
for col in YN:
    df.loc[df[col] =='N', col] = 0
    df.loc[df[col] =='Y', col] = 1
df.fillna(0, inplace =True)

# 把時間做sin/cos轉換，保留原來時間
df = timeSinCosTrans(df)


#### 抓出所有被詐騙過的帳戶進行分析

In [106]:
fraud_df = df.loc[df['fraud_ind'] == 1]
print("總共的詐騙次數共有%d筆"%len(fraud_df))
fraud_bacno_list = list(set(fraud_df['bacno']))

new_df = df[df['bacno'].isin(fraud_bacno_list)]

print("所有被詐騙帳戶的資料有%d筆"%len(new_df))
new_df.head()

總共的詐騙次數共有20355筆
所有被詐騙帳戶的資料有255579筆


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,TIME_SIN,TIME_COS,time
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,457,59333,0,0,102,0,516056,-0.989568,-0.144068,62812.0
16,6716,150887,4541,930.31,5,62,0,4,0,0,...,329,37560,0,5820,102,0,1765957,-0.575243,-0.817982,51628.0
19,6881,152257,65948,513.80,5,0,0,0,0,0,...,457,68930,0,0,102,0,485608,-0.819110,-0.573636,56399.0
23,6881,161031,145047,683.25,5,0,0,0,0,0,...,457,86584,0,0,102,0,486692,-0.086214,-0.996277,44387.0
30,6769,21246,88760,500.14,5,62,0,4,0,0,...,292,79111,0,5817,102,0,395453,-0.959744,0.280876,68715.0


In [107]:
from sklearn.model_selection import train_test_split

# df = new_df.copy()
y = new_df.pop('fraud_ind')
X_data, X_test, y_data, y_test = train_test_split(new_df, y, train_size = .9, stratify = y, random_state = 20)



In [108]:
# 訓練模型評估結果
def train():
    from sklearn.model_selection import StratifiedKFold 
    import xgboost as xgb
    from sklearn.metrics import f1_score
    import sklearn.model_selection


    skf = StratifiedKFold(n_splits=2, random_state=123, shuffle=True)
    skf.get_n_splits(X_data, y_data)
    print(skf)  



    for train_index, test_index in skf.split(X_data, y_data):

        print("TRAIN:", train_index, "TEST:", test_index)  
        X_train, X_valid = X_data[X_data.index.isin(train_index)], X_data[X_data.index.isin(test_index)]
        y_train, y_valid = y_data[y_data.index.isin(train_index)], y_data[y_data.index.isin(test_index)]

    #     print(X_train)
        model = xgb.XGBClassifier(max_depth= '8', learning_rate=.05, n_estimators = 1000,\
                              tree_method = 'gpu_hist', predictor = 'gpu_predictor')

        model.fit(X=X_train,y=y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose = False)

        y_true = y_valid.values
        y_pred = model.predict(X_valid)
        print(f1_score(y_true, y_pred, average=None))

### 訓練模型評估結果

In [109]:
# 提交
#submit()

### 

### 分析
1. 有詐騙的帳戶漢沒有詐騙的帳戶之間的異同 (麻煩)
2. 被詐騙的帳戶資料之間的變化

In [110]:
# 先合併一下X和Y方便分析，順便把index拔掉
data = pd.concat([X_data, y_data], axis = 1)
data.reset_index(inplace = True)

In [111]:
data.columns

Index(['index', 'acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg',
       'etymd', 'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'TIME_SIN', 'TIME_COS', 'time', 'fraud_ind'],
      dtype='object')

### 分析報告
1. 時間沒什麼用, 大部分詐欺的發生點跟平常消費的時段差不多
2. 地點可能有用，但也許透過帳戶而不是卡號來分析地點會更有用 (scity), 可以加入一個消費地點屬於陌生地區的標記
3. 詐欺往往發生在同一個日期上，可以透過單日消費次數來判斷有詐欺風險
4. 3dsmk好像有用，不過很少被使用的樣子
5. 詐欺的總價有時是連續的小筆(金額相同或相似)(在相同日期)，也會出現明顯離群的大筆支出

In [112]:
#從卡號觀察每張卡被刷的 單日總次數/ 單日平均次數

cano_list = list(set(X_data['cano']))
test = cano_list[:0]

for i in test:
    # i是依序被從cano_list中讀出來的卡號, tmpdf則是依據卡號抓取出來的資料，也就是分析對象(每一張卡的所有資料)
    tmp_df = data[data['cano'] == i]
    lis = ['locdt', 'cano', 'conam', 'fraud_ind', 'scity', 'hcefg', 'loctm', 'flg_3dsmk']
    print('\n----------',i)
    print(tmp_df[lis])

### 根據上面的分析做一些特徵工程

#### 先做一個等一下要用來填充特徵工程資料的的column, 'home_scity'

In [113]:
# data.head()

def add_column(data, column_name):
    a = list(data.columns)
    if any(column_name in s for s in a):
        print("Column \'%s\' is already existed"%column_name)
    else:
        a = data.columns[1]
        tmp_df = data[a]
        tmp_df = tmp_df.rename(columns={a: column_name})
        data[column_name] = tmp_df
        
add_column(data, 'total_times')
add_column(data, 'home_scity')
data.head()


,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,...,scity,stocn,stscd,txkey,TIME_SIN,TIME_COS,time,fraud_ind,total_times,home_scity
0,1491756,6716,101179,63962,1430.12,5,62,0,4,0,...,5820,102,0,1310512,-0.997669,0.068233,65739.0,0,6716,6716
1,851651,6032,46244,60653,652.61,5,62,1,2,0,...,5817,102,0,263093,-0.996233,0.086721,65994.0,0,6032,6032
2,557431,3348,16511,54688,216.12,5,62,1,8,0,...,3460,46,0,642641,-0.998980,0.045145,65421.0,0,3348,3348
3,1312760,6292,27587,167725,1443.52,5,62,1,2,0,...,5817,102,0,1531341,-0.681732,0.731602,76085.0,1,6292,6292
4,334490,6767,39287,78821,899.73,5,62,0,5,0,...,5817,102,0,995924,-0.240793,-0.970577,46544.0,0,6767,6767


In [114]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(arr)
np.delete(arr, [1,2], 0)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


array([[1, 2, 3, 4]])

In [122]:
# 增加交易總次數
lis = []
test = cano_list[:]
# tmp_data = data.copy()
data_array = np.array(data)
data_column = list(data.columns)
cnt = 0

for i in test:
    cnt+=1
#     print(len(data))
    
    # i是一個一個被從cano_list中讀出來的卡號
    # a0表示這個卡號總共出現過幾次，也就是總共的交易次數
    a0 = len(tmp_data[tmp_data['cano'] == i])
    
    # col是在所有的column中尋找指定column('total_times')的index
    col = list(tmp_data.columns).index('total_times')
    index = data[data['cano'] == i].index

    data_array[index ,col] = a0

    # 弄一個進度條
    print(cnt/len(cano_list))

pd.DataFrame(data_array, columns = data_column)

5.3140610054203425e-05
0.00010628122010840685
0.00015942183016261026
0.0002125624402168137
0.0002657030502710171
0.0003188436603252205
0.00037198427037942394
0.0004251248804336274
0.0004782654904878308
0.0005314061005420342
0.0005845467105962377
0.000637687320650441
0.0006908279307046445
0.0007439685407588479
0.0007971091508130513
0.0008502497608672548
0.0009033903709214582
0.0009565309809756616
0.001009671591029865
0.0010628122010840685
0.001115952811138272
0.0011690934211924754
0.0012222340312466786
0.001275374641300882
0.0013285152513550856
0.001381655861409289
0.0014347964714634925
0.0014879370815176957
0.0015410776915718992
0.0015942183016261027
0.0016473589116803061
0.0017004995217345096
0.0017536401317887129
0.0018067807418429163
0.0018599213518971198
0.0019130619619513233
0.0019662025720055265
0.00201934318205973
0.0020724837921139334
0.002125624402168137
0.0021787650122223404
0.002231905622276544
0.0022850462323307473
0.0023381868423849508
0.002391327452439154
0.00244446806249

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,...,scity,stocn,stscd,txkey,TIME_SIN,TIME_COS,time,fraud_ind,total_times,home_scity
0,1491756.0,6716.0,101179.0,63962.0,1430.12,5.0,62.0,0.0,4.0,0.0,...,5820.0,102.0,0.0,1310512.0,-0.997669,0.068233,65739.0,0.0,26.0,6716.0
1,851651.0,6032.0,46244.0,60653.0,652.61,5.0,62.0,1.0,2.0,0.0,...,5817.0,102.0,0.0,263093.0,-0.996233,0.086721,65994.0,0.0,201.0,6032.0
2,557431.0,3348.0,16511.0,54688.0,216.12,5.0,62.0,1.0,8.0,0.0,...,3460.0,46.0,0.0,642641.0,-0.998980,0.045145,65421.0,0.0,3.0,3348.0
3,1312760.0,6292.0,27587.0,167725.0,1443.52,5.0,62.0,1.0,2.0,0.0,...,5817.0,102.0,0.0,1531341.0,-0.681732,0.731602,76085.0,1.0,20.0,6292.0
4,334490.0,6767.0,39287.0,78821.0,899.73,5.0,62.0,0.0,5.0,0.0,...,5817.0,102.0,0.0,995924.0,-0.240793,-0.970577,46544.0,0.0,8.0,6767.0
5,4623.0,3946.0,7067.0,203824.0,513.80,5.0,0.0,0.0,0.0,0.0,...,0.0,102.0,0.0,632756.0,-0.281155,-0.959662,47119.0,0.0,1.0,3946.0
6,145347.0,5666.0,99169.0,9558.0,407.87,4.0,62.0,0.0,2.0,0.0,...,5817.0,102.0,0.0,664247.0,0.883186,-0.469022,28313.0,0.0,9.0,5666.0
7,1218745.0,6769.0,138695.0,101538.0,1595.39,5.0,62.0,1.0,8.0,0.0,...,5817.0,102.0,0.0,200407.0,0.181950,-0.983308,40684.0,0.0,10.0,6769.0
8,603839.0,6716.0,9390.0,97156.0,682.99,5.0,62.0,0.0,5.0,0.0,...,5820.0,102.0,0.0,1132068.0,-0.345844,-0.938292,48056.0,0.0,16.0,6716.0
9,152886.0,3191.0,67843.0,4321.0,1266.89,5.0,26.0,0.0,4.0,0.0,...,3240.0,52.0,0.0,757030.0,-0.040713,-0.999171,43760.0,0.0,52.0,3191.0


In [ ]:
np.savez_compressed('modify_data', data)

In [51]:
#從卡號觀察每張卡被刷的月總次數/ 月平均單日次數/ 有刷卡日子的平均次數/ 單日總次數/ 月平均頻率/ 單日最高/ 單日最高頻率
lis = []
test = cano_list[:10]
tmp_data = data.copy()
for i in test:
    # i是一個一個被從cano_list中讀出來的卡號
    tmp_df = tmp_data[tmp_data['cano'] == i]
#     print(tmp_df)
    tmp_data.drop(index = tmp_df.index, inplace = True)
    # 總次數
    a0 = len(tmp_df)
    
    lis_tmp = list(set(tmp_df['total_times']))
#     tmp_data.drop(tmp_data[tmp_data['cano'] == i], inplace = True)
#     print(tmp_data[tmp_data['cano'] == i].index)
#     tmp_data['total_times'].replace()
#     print(tmp_data[tmp_data['cano'] == i])
    for a in lis_tmp:
        
        b = tmp_df[tmp_df['total_times'] ==  a].index
#         print(b.values)
#         print(a,a0)
        tmp_df.loc[b, 'total_times'] = a0
#         tmp_df.loc[b.values,'total_times'] = a0
#         tmp_df.loc[:]['total_times'] = a0
#         tmp_data[tmp_data['cano'] == i].loc[tmp_data['cano'] != a0,'col2'] = a0
#         df.loc[df['col1'] !=' pre','col2']=Nonpre
    print(tmp_df['total_times'])
#     print(lis_tmp)
    # 月平均單日次數
        # set是尋找元素, 在資料集的 locdt也就是日期欄位尋找元素相當於找出在資料集中所有單獨的日子，
        # 也就是資料集中的每個日期 (like 1~30) 
        # len則是知道這個set裡面的元素個數，也就是一個月有幾天
#     a1 = a0/ len(set(data['locdt'])) 
    
#     # 月平均頻率
#     lis_day = list(set(tmp_df['locdt']))
#     for _ in lis_day:
#         # 單日有幾筆
# #         len(tmp_df[tmp_df['locdt'] == _])
#         lis_day.append( len(tmp_df[tmp_df['locdt'] == _]) )
#     for i in 
#     tmp_data[tmp_data['cano'] == i]['total_times'] = a0
#     print(tmp_data[tmp_data['cano'] == i])
#     print(tmp_data[tmp_data['cano'] == i]['total_times'])
#     # 三月單日頻率
#     a2 = a1
    # 第一位的資料是卡號
    lis.append([i, a0])
#     X_data[X_data['']]
print(lis)

# lis[0][1]

3014      12
4399      12
39945     12
78745     12
78795     12
83485     12
88492     12
114451    12
135201    12
160522    12
178064    12
199444    12
Name: total_times, dtype: int64
20346     18
20819     18
27782     18
60367     18
69467     18
70400     18
74386     18
79504     18
80501     18
100251    18
108178    18
136624    18
173439    18
184551    18
191239    18
193160    18
198408    18
229097    18
Name: total_times, dtype: int64
26181     3
87116     3
197891    3
Name: total_times, dtype: int64
145295    1
Name: total_times, dtype: int64
7222      12
8447      12
14849     12
45289     12
55940     12
71666     12
85037     12
103854    12
110919    12
177748    12
206388    12
211308    12
Name: total_times, dtype: int64
16436     6
54873     6
68373     6
80167     6
183277    6
194681    6
Name: total_times, dtype: int64
109143    4
115804    4
117114    4
169513    4
Name: total_times, dtype: int64
52269     4
103839    4
128994    4
141000    4
Name: total_ti

In [ ]:
set(a['locdt'])
len(set(a['locdt']))

lis_locdt=list(set(a['locdt']))
print(lis_locdt)

In [ ]:
lis_bac_day = []

In [ ]:

for i in lis_locdt:
#     print(i) # i = 日期
    locdt_i = a[ a['locdt']==i] 
    #print(len(locdt_i)) #當天交易數量
    #lis_bac_day.append(len(locdt_i))
    lis_bac_day.append([i, len(locdt_i)])

In [ ]:
lis_bac_day

In [ ]:
a[a['locdt'] == 90].index
y[y.index.isin(a[a['locdt'] == 90].index)]

In [ ]:
df_bac_loc = a.copy()

In [ ]:
a.head()

In [ ]:
df_bac_loc.head()

In [ ]:
# df_bac_loc = df_bac_loc.pop('locdt')

In [ ]:
df_bac_loc.loc['acqic']